# <div align="center">unittest — Unit testing framework</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


The ***unittest*** unit testing framework was originally inspired by ***JUnit*** and has a similar flavor as major unit testing frameworks in other languages. It supports test automation, sharing of setup and shutdown code for tests, aggregation of tests into collections, and independence of the tests from the reporting framework.

To achieve this, unittest supports some important concepts in an object-oriented way:

* ***test fixture*** - A test fixture represents the preparation needed to perform one or more tests, and any associate cleanup actions. This may involve, for example, creating temporary or proxy databases, directories, or starting a server process.


* ***test case*** - A test case is the individual unit of testing. It checks for a specific response to a particular set of inputs. unittest provides a base class, TestCase, which may be used to create new test cases.


* ***test suite*** - A test suite is a collection of test cases, test suites, or both. It is used to aggregate tests that should be executed together.


* ***test runner*** - A test runner is a component which orchestrates the execution of tests and provides the outcome to the user. The runner may use a graphical interface, a textual interface, or return a special value to indicate the results of executing the tests.

### Basic example
The unittest module provides a rich set of tools for constructing and running tests. This section demonstrates that a small subset of the tools suffice to meet the needs of most users.

Here is a short script to test three string methods:

In [9]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

# if __name__ == '__main__':
#     unittest.main()

A testcase is created by subclassing unittest.TestCase. The three individual tests are defined with methods whose names start with the letters ***test***. This naming convention informs the test runner about which methods represent tests.

The crux of each test is a call to ***assertEqual()*** to check for an expected result; ***assertTrue()*** or ***assertFalse()*** to verify a condition; or ***assertRaises()**** to verify that a specific exception gets raised. These methods are used instead of the assert statement so the test runner can accumulate all test results and produce a report.

The ***setUp()*** and ***tearDown()*** methods allow you to define instructions that will be executed before and after each test method. They are covered in more detail in the section Organizing test code.


In [10]:
class Perceptron:
    
    def __init__(self, learning_rate=0.01, iterations=10):
        self.learning_rate = learning_rate
        self.iterations    = iterations
        
        self._weights      = None
        self._errors       = []
        self._threshold    = 0.0

    
    def fit(self, training_vector, target_value):
        """Fit training data
        
        Parameters
        ----------
        training_vector : {array-like}, shape = [samples, features]
        target_value    : {array-life}, shape = [samples]
        """
        numberOfSamples, numberOfFeatures = training_vector.shape
        self._weights = np.zeros(numberOfFeatures)
        
        for _ in range(self.iterations):
            errors = 0
            for observation, target in zip(training_vector, target_value):
                update          = self.learning_rate * (target - self.predict(observation))
                self._weights  += update * observation
                self._threshold = update
                errors += int(update != 0.0)
            self._errors.append(errors)
            
    def _net_input(self, training_vector):
        """Calculate net input"""
        return np.dot(training_vector, self._weights)
    
    def predict(self, training_vector):
        """Return class label after unit step
        Given an array of features, returns an array with 
        """
        #This is a confusing line. Explanation: 
        #np.where will first return indices of elements for which the net input is 
        #greater than the threshold. It will then mark them with 1 for match,
        #or -1 for not a match.
        # E.g  INPUT : training_vector = [10,2, -1, -2], threshold = 0
        #      RETURN: np.array([1,1,-1,-1])
        return np.where(self._net_input(training_vector)>= self._threshold, 1, -1)

In [11]:
#Testing
import unittest
class TestPreceptron(unittest.TestCase):
    
    def test_initialization(self):
        
        a_perceptron = Perceptron(learning_rate=0.5, iterations=10)
        self.assertIsNotNone(a_perceptron)
        self.assertEqual    (a_perceptron.learning_rate, 0.5)
        self.assertEqual    (a_perceptron.iterations   ,  10)
    
    def test_net_input(self):
         
        a_perceptron    = Perceptron()
        training_vector = pd.DataFrame({'feature1':[1,1], 
                                        'feature2':[0,1]})
        a_perceptron._weights = [0,1]
        self.assertTrue(np.array_equal(
                        a_perceptron._net_input(training_vector), 
                        [0,1]))
            
        a_perceptron._weights = [2,2]
        self.assertTrue(np.array_equal(
                        a_perceptron._net_input(training_vector), 
                        [2,4]))
        
    def test_predict(self):
        
        a_perceptron    = Perceptron()
        
        columns = ['Classification', 'FeatureA', 'FeatureB']
        data    = {'obs1': [ 1,0,1],
                   'obs2': [ 1,0,1],
                   'obs3': [-1,1,0],
                   'obs4': [-1,1,0]
                   }
        training_set = pd.DataFrame.from_items(data.items(),
                                               orient='index',
                                              columns=columns)
        a_perceptron._weights    = [0,1]
        a_perceptron._threshold  = 1.0 
        #since all the weight is on Feature B the prediction must 
        #equal the original classification
        self.assertTrue(np.array_equal(
                         a_perceptron.predict(
                         training_set[['FeatureA', 'FeatureB']]),
                         training_set['Classification']))
        
    def test_fit(self):
        
        a_perceptron = Perceptron()
        columns = ['Classification', 'FeatureA', 'FeatureB']
        data    = {'obs1': [ 1,0,1],
                   'obs2': [ 1,0,1],
                   'obs3': [-1,1,0],
                   'obs4': [-1,1,0]
                   }
        training_set = pd.DataFrame.from_items(data.items(),
                                               orient='index',
                                              columns=columns)
        
        a_perceptron.fit(training_set[['FeatureA', 'FeatureB']].values, 
                         training_set['Classification'].values)
        
        self.assertEqual(a_perceptron.predict([0,10]),[1])
        self.assertEqual(a_perceptron.predict([10,0]),[-1])
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestPreceptron)
unittest.TextTestRunner(verbosity=4).run(suite)

test_fit (__main__.TestPreceptron) ... ERROR
test_initialization (__main__.TestPreceptron) ... ok
test_net_input (__main__.TestPreceptron) ... ERROR
test_predict (__main__.TestPreceptron) ... ERROR

ERROR: test_fit (__main__.TestPreceptron)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-0dacc77e21ca>", line 58, in test_fit
    training_set = pd.DataFrame.from_items(data.items(),
NameError: name 'pd' is not defined

ERROR: test_net_input (__main__.TestPreceptron)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-0dacc77e21ca>", line 15, in test_net_input
    training_vector = pd.DataFrame({'feature1':[1,1],
NameError: name 'pd' is not defined

ERROR: test_predict (__main__.TestPreceptron)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-11-0dac

<unittest.runner.TextTestResult run=4 errors=3 failures=0>

In [12]:
import numpy as np
import pandas as pd

In [13]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestPreceptron)
unittest.TextTestRunner(verbosity=4).run(suite)

test_fit (__main__.TestPreceptron) ... C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
ok
test_initialization (__main__.TestPreceptron) ... ok
test_net_input (__main__.TestPreceptron) ... ok
test_predict (__main__.TestPreceptron) ... C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
ok

----------------------------------------------------------------------
Ran 4 tests in 0.015s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>